In [ ]:
#---------------------Move the file---------------------------#

In [49]:
# Case 1 Excel Cell Unmerging   
from openpyxl.utils import range_boundaries
from openpyxl import load_workbook
import os

source_folder_path = r"Input_Location\White Paper"

files=os.listdir(source_folder_path)
 
for item in files:
    if item.endswith((".xlsx", ".xlsm")):
        wb = load_workbook(source_folder_path+"\\"+item)
        sheet = wb.worksheets[0]
        filename=item.split(".")
        max_column_index = sheet.max_column
        
        merge_list = []
        for merge in sheet.merged_cells.ranges:
            merge_list.append(merge)

        for group in merge_list:
            #print("Changing merged cells " + str(group.coord))
            min_col, min_row, max_col, max_row = range_boundaries(group.coord)
            top_left_cell_value = sheet.cell(row=min_row, column=min_col).value
            sheet.unmerge_cells(str(group))
            for row in sheet.iter_rows(min_col=min_col, min_row=min_row, max_col=max_col, max_row=max_row):
                for cell in row:
                    cell.value = top_left_cell_value
                    
        wb.save(r"Output_Location\White Paper\PreProcess\\"+filename[0]+".xlsx")
        print(filename[0])


ORDERNUMBER_10100
ORDERNUMBER_10101
ORDERNUMBER_10103
ORDERNUMBER_10244
ORDERNUMBER_10425


In [50]:
# Case 2 Excel Value Extraction
import os
from openpyxl import load_workbook

def process_excel_file(file_path, target_labels,output_folder_path):
    # Load the Excel workbook
    wb = load_workbook(filename=file_path)
    
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        row_index = 0
        col_index=0
        next_cell_value=0
        
        # Iterate through all cells in the sheet
        for row in ws.iter_rows(min_row=1, max_col=ws.max_column, max_row=ws.max_row):
            for cell_index, cell in enumerate(row):
                # Check if the cell value contains any of the target labels
                if any(label.lower() in str(cell.value).lower() for label in target_labels):
                    print("Found target label:", cell.value)
                    global kj
                    
                    kj += 1
                    row_index=cell.row
                    col_index=cell.column
                    # Check if the current cell is not the last column
                    if cell_index < len(row) - 1:
                        next_cell_value = row[cell_index + 1].value
                        if next_cell_value is not None:
                            write_value_in_excel(file_path, next_cell_value, ws.max_column+kj, cell.row, ws.max_row,output_folder_path)
                           
                        else:
                            print("No value found after the target label.")
                    else:
                        print("No value found after the target label.")
        
def write_value_in_excel(file_path,next_cell_value,col_index,row_index,max_row,output_folder_path):
    wb = load_workbook(filename=file_path)
    
    # Assuming the first sheet is the target sheet
    sheet = wb.active    
    filename=item.split(".")
    # Iterate through rows and write the extracted values to desired cells
    for current_row_index in range(row_index, max_row + 1):
        last_column = col_index  # Assuming you want to write the value in the column next to the target label
        cell = sheet.cell(row=current_row_index, column=last_column)
        cell.value = next_cell_value
        #print("Writing value:", next_cell_value, "to row:", current_row_index, "column:", last_column)
    wb.save(os.path.join(output_folder_path, item))
    print(filename[0])

source_folder_path = r"Input_Location\White Paper\PreProcess"
output_folder_path = r"Output_Location\White Paper\PreProcess"
target_labels = ['target_label_1','target_label_2']#Example target_labels = ['order number','order date']
kj=0
files = os.listdir(source_folder_path)

for item in files:
    if item.endswith((".xlsx", ".xlsm")):
        file_path = os.path.join(source_folder_path, item)
        process_excel_file(file_path, target_labels,output_folder_path)


Found target label: ORDER NUMBER:
ORDERNUMBER_10100
Found target label: ORDER DATE:
ORDERNUMBER_10100
Found target label: ORDER NUMBER
ORDERNUMBER_10101
Found target label: ORDER DATE :
ORDERNUMBER_10101
Found target label: ORDER NUMBER:
ORDERNUMBER_10103
Found target label: ORDER DATE:
ORDERNUMBER_10103
Found target label: ORDER NUMBER:
ORDERNUMBER_10244
Found target label: ORDER DATE:
ORDERNUMBER_10244
Found target label: ORDER NUMBER:
ORDERNUMBER_10425
Found target label: ORDER DATE:
ORDERNUMBER_10425


In [51]:
# Case 3 process, write & extract files
import os
import re
from openpyxl import load_workbook

def extract_vehicle_names(sheet, product_code_row, product_code_column):
    names = []
    for row in sheet.iter_rows(min_row=1, min_col=1, max_row=product_code_row, max_col=product_code_column):
        for cell in row:
            if cell.value:
                original_str = str(cell.value)
                str_split = original_str.split(":")
                result = re.sub(r'\s+', ' ', str_split[0])
                new_split = str(result).split(" ")
                if 'x' in str(new_split[0]):
                    names.append(new_split[1])
    return names

def write_vehicle_info(sheet, start_row, vehicle_names):
    last_column = sheet.max_column 
    for row_index in range(start_row, sheet.max_row + 1):
        for idx, name in enumerate(vehicle_names, start=1):
            sheet.cell(row=row_index, column=last_column + idx).value = name

def process_excel_files(input_folder_path, output_folder_path, vehicle_names_dict,reference_value):
    files = os.listdir(input_folder_path)
    
    for item in files:
        if item.endswith((".xlsx", ".xlsm")):
            wb = load_workbook(os.path.join(input_folder_path, item))
            sheet = wb.active
            
            product_code_row = 0
            product_code_column = 0
            
            for row in sheet.iter_rows(min_row=1, min_col=1, max_row=sheet.max_row, max_col=sheet.max_column):
                for cell in row:
                    if cell.value == reference_value:
                        product_code_row = cell.row
                        product_code_column = cell.column
            
            vehicle_names = extract_vehicle_names(sheet, product_code_row, product_code_column)
            
            # Check if vehicle name exists in the dictionary and assign the corresponding value
            vehicle_values = [vehicle_names_dict.get(name, "") for name in vehicle_names]
            write_vehicle_info(sheet, product_code_row, vehicle_values)
            sheet.delete_rows(1, amount=product_code_row-1)
               
            wb.save(os.path.join(output_folder_path, item))

# Paths for input and output folders
input_path = r"Input_Location\White Paper\PreProcess"
output_path = r"Output_Location\White Paper\PreProcess"
# Define category names and their corresponding codes in a dictionary
vehicle_names_dict = {
    "1": "Category 1",
    "2": "Category 2",
    "3": "Category 3",
    "4": "Category 4",
    "5": "Category 5",
    "6": "Category 6"
}
#Example vehicle_names_dict = {  "1": "Motorcycles",    "2": "Classic Cars",    "3": "Vintage Cars",    "4": "Auto",
    #"5": "Trucks",    "6": "Buses"}
reference_value = "reference_name"
#Example reference_value = "PRODUCT CODE"
process_excel_files(input_path, output_path, vehicle_names_dict,reference_value)



In [52]:
# Case 4 Deleting Empty Rows and Columns
import pandas as pd
from openpyxl import load_workbook
import os

def delete_empty_rows_and_columns(input_path, output_path,reference_column):
    # Iterate over each file in the input path
    for filename in os.listdir(input_path):
        if filename.endswith('.xlsx'):
            # Load the Excel spreadsheet
            filepath = os.path.join(input_path, filename)
            wb = load_workbook(filepath)
            ws = wb.active
                
            # Convert the worksheet to a DataFrame
            data = ws.values
            columns = next(data)
            df = pd.DataFrame(data, columns=columns)

            # Remove empty rows
            df = df.dropna(axis=0, how='all')

            # Remove empty columns
            df = df.dropna(axis=1, how='all')
            
            # Remove rows where the first column is empty
            df = df[df.iloc[:, 0].notna()]
            
            # Save the modified DataFrame back to an Excel file
            output_filepath = os.path.join(output_path, filename)
            df.to_excel(output_filepath, index=False)

            print(f"{filename} processed and saved as modified_{filename}")

input_folder = r"Input_Location\White Paper\PreProcess"
output_folder = r"Output_Location\White Paper\PreProcess"
reference_column = 1  # You need to provide the column number where to check for "Customer details"

delete_empty_rows_and_columns(input_folder, output_folder,reference_column)


ORDERNUMBER_10100.xlsx processed and saved as modified_ORDERNUMBER_10100.xlsx
ORDERNUMBER_10101.xlsx processed and saved as modified_ORDERNUMBER_10101.xlsx
ORDERNUMBER_10103.xlsx processed and saved as modified_ORDERNUMBER_10103.xlsx
ORDERNUMBER_10244.xlsx processed and saved as modified_ORDERNUMBER_10244.xlsx
ORDERNUMBER_10425.xlsx processed and saved as modified_ORDERNUMBER_10425.xlsx


In [53]:
# Case 5 Delete rows with specific value & Modify headers
from openpyxl import load_workbook
import os


def process_excel_files(input_folder, output_folder, reference_value, value_to_delete, new_header_values):
    # Iterate over each file in the input path
    for filename in os.listdir(input_folder):
        if filename.endswith((".xlsx", ".xlsm")):
            wb = load_workbook(os.path.join(input_folder, filename))
            sheet = wb.worksheets[0]
            print(filename)
            delete_rows=0

            # Get the maximum row and column numbers
            max_row = sheet.max_row
            max_column = sheet.max_column
            Reference_row = 0
            Reference_column = 0

            # Find the reference row and column
            for row in sheet.iter_rows(min_row=1, min_col=1, max_row=max_row, max_col=max_column):
                for cell in row:
                    if cell.value == reference_value:
                        Reference_row = cell.row
                        Reference_column = cell.column
                        break
                if Reference_row != 0:
                    break

            # Delete rows with specific value
            for row in sheet.iter_rows(min_row=1, min_col=1, max_row=max_row, max_col=Reference_column):
                for cell in row:
                    cell_value = str(cell.value)
                    if value_to_delete.lower() in cell_value.lower():
                        delete_rows=int(cell.row)

            if(delete_rows!=0):
                sheet.delete_rows(delete_rows) 
            delete_row2_1=0

            # Modify headers
            for idx, header_value in enumerate(new_header_values, start=1):
                sheet.cell(row=Reference_row, column=Reference_column + idx).value = header_value

            # Save the modified Excel spreadsheet
            wb.save(os.path.join(output_folder, filename))
            print(f"{filename} processed and saved as {filename}")

input_folder = r"Input_Location\White Paper\PreProcess"
output_folder = r"Output_Location\White Paper\PreProcess\Output\Step1"

reference_value = "reference_name" #Example reference_value = "PRODUCT CODE"
value_to_delete = "Delete rows with specific value"#Example value_to_delete = "Customer details" # Delete rows with specific value
new_header_values = ["header_value_1", "header_value_2"] #Example new_header_values = ["ORDER NUMBER", "ORDER DATE"] #Modify headers

process_excel_files(input_folder, output_folder, reference_value, value_to_delete, new_header_values)

 

ORDERNUMBER_10100.xlsx
ORDERNUMBER_10100.xlsx processed and saved as ORDERNUMBER_10100.xlsx
ORDERNUMBER_10101.xlsx
ORDERNUMBER_10101.xlsx processed and saved as ORDERNUMBER_10101.xlsx
ORDERNUMBER_10103.xlsx
ORDERNUMBER_10103.xlsx processed and saved as ORDERNUMBER_10103.xlsx
ORDERNUMBER_10244.xlsx
ORDERNUMBER_10244.xlsx processed and saved as ORDERNUMBER_10244.xlsx
ORDERNUMBER_10425.xlsx
ORDERNUMBER_10425.xlsx processed and saved as ORDERNUMBER_10425.xlsx


In [54]:
# Case 6 : Moving Values to Columns
import os
from openpyxl import load_workbook

def move_values_to_columns(source_folder_path, destination_folder_path, target_value):
    files = os.listdir(source_folder_path)

    for item in files:
        if item.endswith((".xlsx", ".xlsm")):
            wb = load_workbook(os.path.join(source_folder_path, item))
            sheet = wb.worksheets[0]
            filename = item.split(".")

            # Find the row number where the target value appears in Column  
            start_row = None
             
            # Find the reference row and column
            for row in sheet.iter_rows(min_row=1, min_col=1, max_row=sheet.max_row, max_col=sheet.max_column):
                for cell in row:
                    if cell.value == target_value:
                        start_row = cell.row + 1
                    break
                    
            # Move values dynamically to specified columns
            if start_row:
                Column_Position=sheet.max_column+1
                for i, row in enumerate(range(start_row, sheet.max_row + 1), start=1):
                    values_to_move = [
                        sheet.cell(row=row, column=col).value for col in range(1, sheet.max_column + 1)
                    ]
                    # Write values to specified columns dynamically
                    for j, value in enumerate(values_to_move, start=Column_Position):
                        sheet.cell(row=i, column=j, value=value)
                
            # Delete rows containing the target value
            for row in sheet.iter_rows(min_row=1, min_col=1, max_row=sheet.max_row, max_col=sheet.max_column):
                for cell in row:
                    if target_value.lower() in str(cell.value).lower():
                        sheet.delete_rows(cell.row, amount=sheet.max_row)

            # Save the modified Excel file
            wb.save(os.path.join(destination_folder_path, filename[0] + ".xlsx"))


source_folder_path = r"Input_Location\White Paper\PreProcess\Output\Step1"
destination_folder_path = r"Output_Location\White Paper\PreProcess\Output\Step2"

target_value = "target_value_name" #Example target_value = "2.   Order details" # Delete rows containing the target value
#columns_to_move = ["PRICE EACH", "ORDER LINENUMBER", "SALES", "DAYS_SINCE_LASTORDER", "STATUS", "DEAL SIZE"]

move_values_to_columns(source_folder_path, destination_folder_path, target_value)


In [55]:
# Case 7 Processing Files and Filtering Data
import pandas as pd
import os

def process_files(source_folder_path, destination_folder_path,Column_List,filtered_Column_List):
    files = os.listdir(source_folder_path)

    for item in files:
        if item.endswith((".xlsx", ".xlsm")):
            filename = item.split(".")
            df = pd.read_excel(os.path.join(source_folder_path, item))

            Column_List = Column_List

            my_set = set()

            for column_name in df.columns:
                if column_name not in Column_List:
                    my_set.add(column_name)

            filtered_df = pd.DataFrame(columns=filtered_Column_List)

            for element in my_set:
                for index, row in df.iterrows():
                    new_row = {col: '' for col in filtered_df.columns}

                    for column_name, value in row.iteritems():
                        if column_name in Column_List:
                            new_row[column_name] = value
                        elif column_name == element and value == "O":
                            new_row['Country'] = element

                    filtered_df = filtered_df.append(new_row, ignore_index=True)

            # Check if 'Country' column is empty
            empty_country = filtered_df['Country'].str.strip().eq('')

            # Remove rows where 'Country' is empty
            filtered_df = filtered_df[~empty_country]

            # Reset index after dropping rows
            filtered_df.reset_index(drop=True, inplace=True)

            filtered_df.to_excel(os.path.join(destination_folder_path, f"{filename[0]}.xlsx"), index=False)


source_folder_path = r"Input_LocationInput_Location\White Paper\PreProcess\Output\Step2"
destination_folder_path = r"Output_Location\White Paper\PreProcess\Output\Step3"
Column_List = [
                'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6','Column_7', 'Column_8', 'Column_9','Column_10'    
            ]
#Example Column_List = [
               # 'CONTACT LASTNAME', 'CONTACT FIRSTNAME', 'QUANTITY ORDERED', 'CUSTOMER NAME', 'PHONE', 'ADDRESS',
               # 'CITY', 'POSTALCODE', 'PRODUCT CODE', 'ORDER DATE', 'ORDER NUMBER', 'Motorcycles', 'Classic Cars',
               # 'Vintage Cars', 'Auto', 'Trucks', 'Buses', 'PRICE EACH', 'ORDER LINENUMBER', 'SALES',
               # 'DAYS_SINCE_LASTORDER', 'STATUS', 'DEAL SIZE'     ]
filtered_Column_List=[
                  'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6','Column_7', 'Column_8', 'Column_9','Column_10','Category_Column_Name'    
            ]
#Example filtered_Column_List=[
               # 'CONTACT LASTNAME', 'CONTACT FIRSTNAME', 'Country', 'QUANTITY ORDERED', 'CUSTOMER NAME', 'PHONE',
               # 'ADDRESS', 'CITY', 'POSTALCODE', 'PRODUCT CODE', 'ORDER DATE', 'ORDER NUMBER', 'Motorcycles',
               # 'Classic Cars', 'Vintage Cars', 'Auto', 'Trucks', 'Buses', 'PRICE EACH', 'ORDER LINENUMBER',
               # 'SALES', 'DAYS_SINCE_LASTORDER', 'STATUS', 'DEAL SIZE'     ]
process_files(source_folder_path, destination_folder_path,Column_List,filtered_Column_List)


In [56]:
# Case 8 Merging Excel Files and Exporting to CSV
 
import glob
#import pandas as pd
#import datetime
#from datetime import datetime
# specifying the path to csv files1
path = r"Input_Location\White Paper\PreProcess\Output\Step3"

# csv files1 in the path
file_list = glob.glob(path + "/*.xlsx")

# list of excel files1 we want to merge.
# data into pandas dataframe.
excl_list = []

for file in file_list:
    excl_list.append(pd.read_excel(file))

# create a new dataframe to store the
# merged excel file.
excl_merged = pd.DataFrame()

for excl_file in excl_list:

    # appends the data into the excl_merged
    # dataframe.
    excl_merged = excl_merged.append(
    excl_file, ignore_index=True)

excl_merged.to_excel(path+"\Merge_Files.xlsx", index=False,header=True) 
excl_merged = excl_merged.replace("\n"," ",regex=True)
excl_merged = excl_merged.replace("\r"," ",regex=True)
excl_merged = excl_merged.replace("\t"," ",regex=True)
excl_merged = excl_merged.replace("\|"," ",regex=True)

 
#excl_merged['ORDER DATE'] = pd.to_datetime(excl_merged.Date,errors='coerce')

#excl_merged['ORDER DATE'] = excl_merged['ORDER DATE'].dt.strftime('%Y-%m-%d')
excl_merged.to_csv(path+"\Merge_Files.csv",index=False,header=False,sep="|")

print("Program has stopped")


Program has stopped


In [59]:
#Case 9 Data Transformation
#import pandas as pd
#from datetime import datetime

# Load the Excel file into a DataFrame
df = pd.read_excel(r"Input_Location\White Paper\PreProcess\Output\Step3\Merge_Files.xlsx")
# Convert 'Date' column to datetime with the desired format
df['DATE_Column'] = pd.to_datetime(df['DATE_Column'], errors='coerce')#Example df['ORDER DATE'] = pd.to_datetime(df['ORDER DATE'], errors='coerce')
 

# Now, if you want to change the format without converting to and from datetime objects
df['DATE_Column'] = df['DATE_Column'].dt.strftime('%Y-%m-%d') #Example df['ORDER DATE'] = df['ORDER DATE'].dt.strftime('%Y-%m-%d')
df['DATE_Column'] #Example df['ORDER DATE']
df = df.replace("\n"," ",regex=True) 
df = df.replace("\r"," ",regex=True)
df = df.replace("\t"," ",regex=True)
df = df.replace("\|"," ",regex=True)

# Melt the DataFrame to convert "Vintage Cars" and "Auto" columns into rows
df_melted = pd.melt(df, id_vars=['Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6','Column_7', 'Column_8', 'Column_9','Column_10','Category_Column_Name'    
           ],
                    value_vars=['value 1', 'value 2', 'value 3', 'value 4','value 5','value 6'],
                    var_name='value_Type', value_name='value_name')
#Example df_melted = pd.melt(df, id_vars=['CONTACT LASTNAME', 'CONTACT FIRSTNAME', 'Country', 'QUANTITY ORDERED',
                                 # 'CUSTOMER NAME', 'PHONE', 'ADDRESS', 'CITY', 'POSTALCODE',
                                 # 'PRODUCT CODE', 'ORDER DATE', 'ORDER NUMBER','PRICE EACH','ORDER LINENUMBER','SALES','DAYS_SINCE_LASTORDER','STATUS','DEAL SIZE'],
                   # value_vars=['Motorcycles', 'Classic Cars', 'Vintage Cars', 'Auto','Trucks','Buses'],
                   # var_name='Vehicle_Type', value_name='Vehicle')
# Filter rows where 'Vehicle' is not None
df_melted = df_melted[df_melted['value_name'].notna()]

# Save the DataFrame back to Excel
df_melted.to_excel(r"Input_Location\White Paper\PreProcess\Output\Step4\output_file.xlsx", index=False)
df_melted.to_csv(r"Output_Location\White Paper\PreProcess\Output\Step4\output_file.csv",index=False,header=False,sep="|")


In [58]:
#end of the code